In [1]:

import os
import shutil
import random

# Paths
original_dataset = "D:/Capstone Project/IMG_CLASSES - Copy"     # Replace with your actual path
sampled_dataset = "Sampled_IMG_CLASSES_2"
os.makedirs(sampled_dataset, exist_ok=True)

# Target sample count
sample_size = 1500

# Loop through each class folder
for class_name in os.listdir(original_dataset):
    class_path = os.path.join(original_dataset, class_name)
    if not os.path.isdir(class_path):
        continue

    # Output class folder
    output_class_path = os.path.join(sampled_dataset, class_name)
    os.makedirs(output_class_path, exist_ok=True)

    # List and sample/copy images
    images = os.listdir(class_path)
    num_to_sample = min(sample_size, len(images))
    sampled_images = random.sample(images, num_to_sample)

    for img_name in sampled_images:
        src = os.path.join(class_path, img_name)
        dst = os.path.join(output_class_path, img_name)
        shutil.copy2(src, dst)

    print(f"Class '{class_name}': sampled {num_to_sample} images (available: {len(images)})")

print("\nSampling complete. Output folder:", sampled_dataset)


Class '1. Eczema - 1.6k': sampled 1500 images (available: 1677)
Class '2. Melanoma - 3.1k': sampled 1500 images (available: 3140)
Class '3. Atopic Dermatitis - 1.25k': sampled 1257 images (available: 1257)
Class '4. Melanocytic Nevi - 7.9k': sampled 1500 images (available: 7970)
Class '5. Benign Keratosis - 2k': sampled 1500 images (available: 2079)
Class '6. Fungal Infections - 1.7k': sampled 1500 images (available: 1702)
Class '7. Viral Infections - 2.1k': sampled 1500 images (available: 2103)

Sampling complete. Output folder: Sampled_IMG_CLASSES_2


In [3]:
import os
import cv2

# Input and output directories
input_dir = "Sampled_IMG_CLASSES_2"
output_dir = "Resized_IMG_CLASSES_2"
os.makedirs(output_dir, exist_ok=True)

# Target size
target_size = (224, 224)

# Resize and save
for class_name in os.listdir(input_dir):
    class_input_path = os.path.join(input_dir, class_name)
    class_output_path = os.path.join(output_dir, class_name)
    os.makedirs(class_output_path, exist_ok=True)

    for img_name in os.listdir(class_input_path):
        img_path = os.path.join(class_input_path, img_name)
        image = cv2.imread(img_path)

        if image is None:
            print(f" Skipping unreadable image: {img_path}")
            continue

        # Resize image
        resized = cv2.resize(image, target_size)

        # Save
        output_path = os.path.join(class_output_path, img_name)
        cv2.imwrite(output_path, resized)

    print(f" Resized images for class '{class_name}'")

print("\n All images resized to 224x224 and saved in 'Resized_IMG_CLASSES_2'")


 Resized images for class '1. Eczema - 1.6k'
 Resized images for class '2. Melanoma - 3.1k'
 Resized images for class '3. Atopic Dermatitis - 1.25k'
 Resized images for class '4. Melanocytic Nevi - 7.9k'
 Resized images for class '5. Benign Keratosis - 2k'
 Resized images for class '6. Fungal Infections - 1.7k'
 Resized images for class '7. Viral Infections - 2.1k'

 All images resized to 224x224 and saved in 'Resized_IMG_CLASSES_2'


In [5]:
import os
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

# Paths
input_root = "Resized_IMG_CLASSES_2"
output_root = "oversampled_normalized_images"

# Models and their specific normalization transforms
model_transforms = {
    "densenet": transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
    "resnet": transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225]),
    "mobilenetv2": None  # Only /255 scaling, handled separately
}

# Base transformation (resize skipped)
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

# Loop through model types
for model_name, norm_transform in model_transforms.items():
    print(f"Processing for {model_name}...")

    model_output_dir = os.path.join(output_root, model_name)
    os.makedirs(model_output_dir, exist_ok=True)

    # Go through each class folder
    for class_name in os.listdir(input_root):
        class_input_dir = os.path.join(input_root, class_name)
        class_output_dir = os.path.join(model_output_dir, class_name)
        os.makedirs(class_output_dir, exist_ok=True)

        # Loop through each image in the class folder
        for img_name in tqdm(os.listdir(class_input_dir), desc=f"{model_name} - {class_name}"):
            img_path = os.path.join(class_input_dir, img_name)
            try:
                image = Image.open(img_path).convert("RGB")
                tensor = to_tensor(image)

                if norm_transform:
                    tensor = norm_transform(tensor)  # Standard normalization
                else:
                    # MobileNetV2: only scaling to [0, 1], already done by ToTensor
                    pass

                # Convert back to PIL image (scaled for saving)
                norm_image = to_pil(tensor)
                norm_image.save(os.path.join(class_output_dir, img_name), format='JPEG')

            except Exception as e:
                print(f"Error processing {img_path}: {e}")


Processing for densenet...


densenet - 7. Viral Infections: 100%|██████████| 1500/1500 [00:19<00:00, 76.95it/s]


Processing for resnet...


resnet - 7. Viral Infections: 100%|██████████| 1500/1500 [00:08<00:00, 173.54it/s]


Processing for mobilenetv2...


mobilenetv2 - 7. Viral Infections: 100%|██████████| 1500/1500 [00:07<00:00, 197.85it/s]
